<a href="https://colab.research.google.com/github/sigma-asad/hds5210-2023/blob/main/midterm_2023_hds5210.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [1]:
import requests
import json

In [2]:
def priest(sex: str,
           age_in_years: int,
           respiratory_rate: float,
           oxygen_saturation: float,
           heart_rate: float,
           systolic_bp_mmhg: float,
           temperature: float,
           alertness: str,
           inspired_oxygen: str,
           performance_status: str) -> float:

    """
    This function takes totally takes 10 input arguments which are described as
    Input args:
    -------------
        Sex (Gender assigned at birth)
        Age in years
        Respiratory rate in breaths per minute
        Oxygen saturation as a percent between 0 and 1
        Heart rate in beats per minute
        Systolic BP in mmHg
        Temperature in degrees C
        Alertness as a string description
        Inspired Oxygen as as string description
        Performance Status as a string description

    Output:
    -------------
        It returns float value which is the PRIEST Score to 30-day probability.

    here are some test cases and expected results
    --------------
    >>> priest("FEMALE", 66, 19.0, 0.96, 135.0, 115.0, 38.1, "ALERT", "AIR", "bed/chair bound, no self-care")
    0.34

    >>> priest("Male",16,23.0,0.96,135.0,115.0,39.2,"ALERT","supplemental oxygen","unrestricted normal activity")
    0.29

    >>> priest("FEMALE",  40,  23.0,  0.96,  41.0,  115.0,  35.4,  "ALERT",  "AIR",  "unrestricted normal activity")
    0.03

    >>> priest("FEMALE",16,22.0,0.96,135.0,115.0,39.2,"Confused or Not Alert","supplemental oxygen","bed/chair bound, no self-care")
    0.55

    >>> priest("Male",40,12.0,0.93,105.0,91.0,35.1,"ALERT","AIR","limited activity, can self-care")
    0.26
    """

    # converting the sex to lower case to maintain the uniformity
    result_points = 0
    SEX_REGISTRY = {"male": 1, "female": 0}
    sex = sex.lower()
    result_points += SEX_REGISTRY.get(sex, 0)

    # Age point fitter
    if age_in_years >= 50 and age_in_years <= 65:
        result_points += 2
    elif age_in_years >= 66 and age_in_years <= 80:
        result_points += 3
    elif age_in_years > 80:
        result_points += 4

    # respiratory rate fitter
    # TODO : check for proper conditions
    if respiratory_rate < 9:
        result_points += 3
    elif respiratory_rate >= 9 and respiratory_rate <= 11:
        result_points += 1
    elif respiratory_rate >= 12 and respiratory_rate <= 20:
        result_points += 0
    elif respiratory_rate >= 21 and respiratory_rate <= 24:
        result_points += 2
    elif respiratory_rate > 24:
        result_points += 3

    # oxygen saturation fitter
    if oxygen_saturation > 0.95:
        result_points += 0
    elif oxygen_saturation >= 0.94 and oxygen_saturation <= 0.95:
        result_points += 1
    elif oxygen_saturation >= 0.92 and oxygen_saturation <= 0.93:
        result_points += 2
    else:
        result_points += 3

    # heart rate fitter
    if heart_rate < 41:
        result_points += 3
    elif heart_rate >= 41 and heart_rate <= 50:
        result_points += 1
    elif heart_rate >= 51 and heart_rate <= 90:
        result_points += 0
    elif heart_rate >= 91 and heart_rate <= 110:
        result_points += 1
    elif heart_rate >= 111 and heart_rate <= 130:
        result_points += 2
    else:
        result_points += 3

    # systolic bp fitter
    if systolic_bp_mmhg < 91:
        result_points += 3
    elif systolic_bp_mmhg >= 91 and systolic_bp_mmhg <= 100:
        result_points += 2
    elif systolic_bp_mmhg >= 101 and systolic_bp_mmhg <= 110:
        result_points += 1
    elif systolic_bp_mmhg >= 111 and systolic_bp_mmhg <= 219:
        result_points += 0
    else:
        result_points += 3

    # temperature fitter
    if temperature < 35.1:
        result_points += 3
    elif temperature >= 35.1 and temperature <= 36:
        result_points += 1
    elif temperature >= 36.1 and temperature <= 38:
        result_points += 0
    elif temperature >= 38.1 and temperature <= 39:
        result_points += 1
    else:
        result_points += 2

    # alert fitter
    ALERT_REGISTRY = {"alert": 0, "confused or not alert": 3}
    result_points += ALERT_REGISTRY.get(alertness.lower(), 0)

    # inspired oxygen fitter
    OXYGEN_REGISTRY = {"air": 0, "supplemental oxygen": 2}
    result_points += OXYGEN_REGISTRY.get(inspired_oxygen.lower(), 0)

    # performance status fitter
    PERFORMANCE_REGISTRY = {'bed/chair bound, no self-care': 4,
                            'limited activity, can self-care': 2,
                            'limited self-care': 3,
                            'limited strenuous activity, can do light activity': 1,
                            'unrestricted normal activity': 0}
    result_points += PERFORMANCE_REGISTRY.get(performance_status.lower(), 0)

    PRIEST_SCORE_PROBABILITY_TABLE = {0: 0.01, 1: 0.01, 2: 0.02, 3: 0.02,
                                      4: 0.03, 5: 0.09, 6: 0.15, 7: 0.18,
                                      8: 0.22, 9: 0.26,
                                      10: 0.29, 11: 0.34,
                                      12: 0.38, 13: 0.46, 14: 0.47, 15: 0.49, 16: 0.55}
    if result_points<17:
        return PRIEST_SCORE_PROBABILITY_TABLE.get(result_points, 0.1)
    elif result_points>=17 and result_points<=25:
        return 0.59
    return 0.99

In [3]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.10/doctest.py", line 1501, in run
    sys.settrace(save_trace)



Finding tests in NoName
Trying:
    priest("FEMALE", 66, 19.0, 0.96, 135.0, 115.0, 38.1, "ALERT", "AIR", "bed/chair bound, no self-care")
Expecting:
    0.34
ok
Trying:
    priest("Male",16,23.0,0.96,135.0,115.0,39.2,"ALERT","supplemental oxygen","unrestricted normal activity")
Expecting:
    0.29
ok
Trying:
    priest("FEMALE",  40,  23.0,  0.96,  41.0,  115.0,  35.4,  "ALERT",  "AIR",  "unrestricted normal activity")
Expecting:
    0.03
ok
Trying:
    priest("FEMALE",16,22.0,0.96,135.0,115.0,39.2,"Confused or Not Alert","supplemental oxygen","bed/chair bound, no self-care")
Expecting:
    0.55
ok
Trying:
    priest("Male",40,12.0,0.93,105.0,91.0,35.1,"ALERT","AIR","limited activity, can self-care")
Expecting:
    0.26
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [4]:
def find_hospital(age, sex, risk_percentage):
    """
    This function takes totally takes 3 input arguments which are described as
    Input args:
    -------------
        Age in years
        Sex (Gender assigned at birth)
        Risk percentage

    Output:
    -------------
        It returns hospital name .

    here are some test cases and expected results
    --------------
    >>> find_hospital(40, "male", 0.26)
    'Southwest Hospital and Medical Center'

    >>> find_hospital(65, "female", 0.26)
    'Wesley Woods Geriatric Hospital'

    >>> find_hospital(18, "female", 0.18)
    'Select Specialty Hospital - Northeast Atlanta'
    """
    try:
        params = {"age": age, "sex" : sex.lower(), "risk_pct" : risk_percentage}
        URL = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/"
        response = requests.request("GET",URL, params = params)
        if response.status_code >= 200 and response.status_code < 300:
            data = response.json()
            return data.get("hospital")
    except Exception as err:
        print(err)
        return None

In [5]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, "male", 0.26)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(65, "female", 0.26)
Expecting:
    'Wesley Woods Geriatric Hospital'
ok
Trying:
    find_hospital(18, "female", 0.18)
Expecting:
    'Select Specialty Hospital - Northeast Atlanta'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [6]:
def get_address(hospital_name):
    """
    This function takes totally takes 3 input arguments which are described as
    Input args:
    -------------
        hospital_name

    Output:
    -------------
        It returns hospital address

    here are some test cases and expected results
    --------------
    >>> get_address("Wesley Woods Geriatric Hospital")
    '1821 CLIFTON ROAD, NE'

    >>> get_address("Select Specialty Hospital - Northeast Atlanta")
    '1821 CLIFTON ROAD NE'

    >>> get_address("Southwest Hospital and Medical Center")
    '501 FAIRBURN ROAD SW'
    """
    URL = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
    response = requests.request("GET", URL)
    if response.status_code >= 200 and response.status_code<300:
        data = response.json()
        hospital_data = data.get(hospital_name.upper(), {})
        return hospital_data.get("ADDRESS")

In [7]:
import doctest
doctest.run_docstring_examples(get_address, globals(),verbose=True)

Finding tests in NoName
Trying:
    get_address("Wesley Woods Geriatric Hospital")
Expecting:
    '1821 CLIFTON ROAD, NE'
ok
Trying:
    get_address("Select Specialty Hospital - Northeast Atlanta")
Expecting:
    '1821 CLIFTON ROAD NE'
ok
Trying:
    get_address("Southwest Hospital and Medical Center")
Expecting:
    '501 FAIRBURN ROAD SW'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [8]:
def process_people():
    """
    This function will download the data from URL, as its the REST service.
    Once the data is downloaded we will try to iter each line and preprocess each data with proper type casting
    and then pass it to the priest fuction to get risk percentage
    based on risk percentage we wil try to get the hospital name and address

    Output
    ------------
        Records - these records is a dictionary which contains patitent identity as key
            and dictionary looks like below line.
            { patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness,
            inspired, status, risk_percentage, hospital, address]}
    """
    URL = "https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh"
    response = requests.request("GET", URL)
    if response.status_code>=200 and response.status_code<300:
        data = response.iter_lines()
        header = True
        header_info = []
        records = {}
        int_converting = ["age"]
        float_converting = ["breath","o2 sat","heart","systolic bp","temp"]
        for i in data:
            record = i.decode().split("|")
            if header:
                header_info = record
                header = False
            else:
                temp = {}
                patient_id = record[0]
                for key,value in zip(header_info[1:], record[1:]):
                    if key in int_converting:
                        temp[key] = int(value)
                    elif key in float_converting:
                        temp[key] = float(value)
                    else:
                        temp[key] = value
                risk_pct = priest(temp["sex"],
                                    temp["age"],
                                    temp["breath"],
                                    temp["o2 sat"],
                                    temp["heart"],
                                    temp["systolic bp"],
                                    temp["temp"],
                                    temp["alertness"],
                                    temp["inspired"],
                                    temp["status"])
                temp["risk_pct"] = risk_pct
                hospital_name = find_hospital(temp.get("age"), temp.get("sex"), risk_pct)
                temp["hospital"] = hospital_name
                hospital_address = get_address(hospital_name)
                temp["address"] = hospital_address
                records[patient_id] = list(temp.values())
                print(list(temp.values()))
        return records

In [9]:
data = process_people()

['FEMALE', 40, 24.0, 0.96, 105.0, 115.0, 34.9, 'ALERT', 'AIR', 'unrestricted normal activity', 0.15, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE']
['Female', 51, 19.0, 0.99, 90.0, 91.0, 38.1, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 0.18, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE']
['female', 40, 29.0, 0.96, 105.0, 95.0, 38.1, 'ALERT', 'air', 'unrestricted normal activity', 0.18, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE']
['FEMALE', 66, 19.0, 0.96, 135.0, 115.0, 38.1, 'ALERT', 'AIR', 'bed/chair bound, no self-care', 0.34, 'Wesley Woods Geriatric Hospital', '1821 CLIFTON ROAD, NE']
['MALE', 18, 22.0, 0.95, 105.0, 115.0, 36.3, 'Confused or Not Alert', 'air', 'unrestricted normal activity', 0.22, 'Southwest Hospital and Medical Center', '501 FAIRBURN ROAD SW']
['Male', 16, 23.0, 0.96, 135.0, 115.0, 39.2, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 0.29, 'Childre

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [10]:
URL = "https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5"
response = requests.request("GET", URL)
if response.status_code>=200 and response.status_code<300:
    actual_file_data = response.json()
    # lets start comparing the data here
    for i in data.keys():
        data_value = data[i]
        actual_data_value = actual_file_data[i]
        if data_value != actual_data_value:
            print(i)
            print(data_value, actual_data_value)

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---

In [11]:
def summarize_results(patients):
    """
    This function is used to summarize the results of the patients where
    If the PRIEST probably is less than 10%, change the hospital name to both have a value of 'Home'
    indicating patients with a lower probability of an adverse outcome can just go home rather
    than be admitted to a hospital. this should help us understand how many users are going to hospital or home

    Input
    ---------
        patients :

    Output
    ---------
        response: its the dictionary which consists key as hospital name and value as patient count who is going to that hospital
    """
    response = {}
    for key in patients.keys():
        record = patients[key]
        priest_pct = record[-3]
        if priest_pct < 0.1:
            if "Home" in response:
                response["Home"] += 1
            else:
                response["Home"] = 1
        else:
            hospital_name = record[-2]
            if hospital_name in response:
                response[hospital_name] += 1
            else:
                response[hospital_name] = 1
    return response

In [12]:
summarize_results(data)

{'Select Specialty Hospital - Northeast Atlanta': 32,
 'Wesley Woods Geriatric Hospital': 20,
 'Southwest Hospital and Medical Center': 16,
 'Childrens Healthcare of Atlanta at Scottish Rite': 1,
 'Home': 24,
 'Hughes Spalding Childrens Hospital': 1,
 'Emory Dunwoody Medical Center': 5}